## Space Invaders - AI23

In [2]:
import keras
from keras import layers
import tensorflow as tf
import gymnasium as gym
from gymnasium.wrappers.frame_stack import FrameStack
from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing
import numpy as np
import tensorflow as tf
import ale_py

import pandas as pd
import plotly_express as px

### Här nedan hade vi från början ett epsilon-värde på 1. Nu har vi ändrat epsilon till 0,1 och epsilon-max till 0.11. För att undvika större ändringar i koden och för att inte ta bort epsilon-max helt, valde vi att sätta ett symboliskt värde (0,11). Detta värde påverkar inte modellen särskilt mycket. Eftersom vi tränade modellen över 1.5 milon frames behövde vi justera epsilon-värdet. Ändringen säkerställde att modellen inte utförde helt slumpmässiga handlingar.

In [ ]:
seed = 42
gamma = 0.99
epsilon = 0.1 #epsilon = 1.0
epsilon_min = 0.1
epsilon_max = 0.11 # epsilon = 1.0
epsilon_interval = (epsilon_max - epsilon_min)
batch_size = 32
max_steps_per_episode = 10000
max_episodes = 0
max_frames = 1e7

### Koden nedan skapar själva miljön och sätter en trigger för att spara en videos. Videorna sparas i en mapp som heter "videos". Totalt har jag sparat cirka tio olika videos. Eftersom de flesta av videorna var mer eller mindre lika, valde jag att behålla ungefär tio stycken för att få en representativ översikt. Efter detta skriver koden ut antalet actions som agenten kan utföra. Det visade sig vara sex olika actions. Genom att köra cellen kunde jag också skriva ut en lista över vilka dessa actions är.

In [ ]:
gym.register_envs(ale_py)
env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="rgb_array")
env = AtariPreprocessing(env)

env = FrameStack(env, 4)

trigger = lambda t: t % 1000 == 0
env = gym.wrappers.RecordVideo(env, video_folder="videos", episode_trigger=trigger, disable_logger=True)

num_actions = env.action_space.n
  
action_meanings = env.unwrapped.get_action_meanings()
print(f"Number of actions: {num_actions} \n\nActions: {action_meanings}") 

### Här definieras själva modellen. Ursprungligen användes en lambda-funktion för att hantera inputen, men den har vi tagit bort. Istället hanteras inputen nu av en separat funktion, som vi definierar i nästa cell. Denna funktion ansvarar för att omstrukturera ordningen på inputen så att vi kunde spara modellerna.

In [ ]:
def create_q_model():
    return keras.Sequential(
        [
            layers.Conv2D(32, kernel_size=8, strides=4, activation="relu"),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="relu"),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="relu"),
            layers.Flatten(),
            layers.Dense(512, activation="relu"),
            layers.Dense(num_actions, activation="linear")
        ]
    )

# Input funktionen (4, 84, 84) (84,84,4)

In [ ]:
def preprocess_input(data):
    return np.transpose(data, (1, 2, 0))

### Från början användes en helt tom modell. Efter att ha tränat modellen ett antal gånger sparades den vid varje avslutad träningsomgång. Varje gång datorn startades om, fortsatte vi träningen genom att ladda den senaste sparade modellen. Vid återupptagandet uppdaterades också värden för vilken episod modellen sparades vid, samt antalet frames som kördes fram till dess. Senaste gången sattes modellen för att fortsätta träna var den uppe på 14930 episoder och 9590000 frames. Dessa värden matades in för att säkerställa att träningen kunde fortsätta från rätt punkt.

In [ ]:
model = keras.models.load_model("models/14930.keras")
model_target = keras.models.load_model("models/14930.keras")

optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 14930
frame_count = 9490000

epsilon_random_frames = 1
epsilon_greedy_frames = 1

max_memory_length = 10000
update_after_actions = 4
update_target_network = 10000
loss_function = keras.losses.Huber()

### Här nedan visas själva träningsloopen (trännades ca 8-9 dagar), som i stort sett lämnades oförändrad. Det enda som ändrades var att vi använde funktionen preprocess_input på tre olika ställen för att hantera inputdata. Utöver detta sparades(efter ca 5000 episoder och 2.5 miljoner frames) poängen som modellen uppnådde vid varje episod samt  andra viktiga data, varje gång en modell sparades, inklusive running reward och antal frames. Dessa data användes senare för att skapa grafer som presenteras längre ned i arbetet.

In [ ]:
while True:
    observation, _ = env.reset()
    state = np.array(observation)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1

        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            action = np.random.choice(num_actions)
        else:
            state_processed = preprocess_input(state)
            state_tensor = tf.convert_to_tensor(state_processed)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            action = keras.ops.argmax(action_probs[0]).numpy()

        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        state_next, reward, done, _, _ = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)

        state = state_next

        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            state_next_sample_processed = np.array([preprocess_input(s) for s in state_next_sample])
            future_rewards = model_target.predict(state_next_sample_processed, verbose=0)

            updated_q_values = rewards_sample + gamma * keras.ops.amax(future_rewards, axis=1)
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            masks = keras.ops.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                state_sample_processed = np.array([preprocess_input(s) for s in state_sample])
                q_values = model(state_sample_processed)

                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)

                loss = loss_function(updated_q_values, q_action)

            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if frame_count % update_target_network == 0:
            model_target.set_weights(model.get_weights())
            with open("history/running_rewards.txt", "a") as fil:
                fil.write(f"Best score of last 100: {np.max(episode_reward_history)}, running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}\n")
            print(f"Best score of last 100: {np.max(episode_reward_history)}, running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}")
            model.save(f"models/breakout_qmodel_{episode_count}.keras")

        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1
    with open("history/episod_logg.txt", "a") as fil:
        fil.write(f"Episode {episode_count-1}: {episode_reward}\n")
    print(f"Episode {episode_count-1}: {episode_reward}\n")

    if running_reward > 800:
        print("Solved at episode {}!".format(episode_count))
        break

    if max_episodes > 0 and episode_count >= max_episodes:
        print("Stopped at episode {}!".format(episode_count))
        break
    if max_frames <= frame_count:
        print(f"Stopped at frame {frame_count}!")
        break


# Episode - Score

In [3]:
frames_df = pd.read_csv("history/frames.csv")
episode_df = pd.read_csv("history/episodes.csv")

In [4]:
episode_df.head()

,Episode,Score
0,4750,260.0
1,4751,175.0
2,4752,345.0
3,4753,245.0
4,4754,175.0


#### Max score = 1095
#### Min score = 15

In [9]:
episode_df["Score"].min(), episode_df["Score"].max()

(15.0, 1095.0)

#### Mean = 328.6

In [10]:
episode_df["Score"].mean()

328.6033163265306

### Nedan visas grafen som illustrerar hur mycket poäng modellen fick för varje episod. Grafen är något svår att läsa eftersom den innehåller många datapunkter, vilket gör att man behöver zooma in för att tydligt se poängfördelningen. För att underlätta tolkningen har jag också skrivit ut max, min, och medelvärde för poängen. Dessa sammanfattande värden ger en bättre förståelse för modellens prestanda.

In [5]:
fig = px.line(episode_df, x='Episode', y='Score', title='Score per Episode', markers=True)
fig.write_html("episodes.html")
fig.show()

In [6]:
frames_df.head()

,Running Reward,Frame Count
0,275,2610000
1,271,2620000
2,274,2630000
3,274,2640000
4,277,2650000


#### Max Running Reward = 271
#### Min Running Reward = 383

In [13]:
frames_df["Running Reward"].min(), frames_df["Running Reward"].max()

(271, 383)

#### Mean = 328.67

In [14]:
frames_df["Running Reward"].mean()

328.677027027027

### Här visas en graf där vi, istället för att logga data för varje episod, har sparat Running Reward för varje tiotusende frame. Denna metod gör grafen betydligt tydligare och ger en mer överskådlig bild av modellens utveckling under träningen.

In [7]:
fig = px.line(frames_df, x='Frame Count', y='Running Reward', title='Rewards by Frames', markers=True)
fig.write_html("frames.html")
fig.show()


## Resultat: efter att ha tränat modellen i 15 721 episoder och 10 000 000 frames uppnådde jag ett Running Reward mean på 328 och max på 383 poäng samt max score på 1095.